# Neural Network with 2 hidden layers

### Load Modules

In [1]:
# Set the Random Seed
seed_value= 2022

import os
os.environ['PYTHONHASHSEED']=str(seed_value)
import numpy  as np
import pandas  as pd
import datetime
import random
from sklearn.metrics import mean_squared_error, r2_score
import statsmodels.formula.api as smf
import tensorflow as tf
from keras.models     import Sequential, load_model
from keras.layers     import Activation, Dense, LeakyReLU
from keras.optimizers import Adam
from keras.callbacks  import EarlyStopping, ModelCheckpoint
from keras.regularizers import l1
from tensorflow.keras.losses import MeanSquaredError

session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
tf.compat.v1.keras.backend.set_session(sess)

In [2]:
def reset_seeds(seed_value=2022):
    os.environ['PYTHONHASHSEED']=str(seed_value)
    np.random.seed(seed_value)
    random.seed(seed_value)
    tf.random.set_seed(seed_value)
reset_seeds()

### Load Dataset - include RepRisk data

In [3]:
# Load monthly firm characteristics raw data
df = pd.read_parquet('C:/Users/rafae/Documents/HSG/Master Thesis/Data/Final/data07_model_input.parquet')
df = df.sort_values(by=['YM', 'permno'])
df = df.set_index(['year', 'YM', 'permno'])
df

gvkey  reprisk_id         prc         vol         mve_m  \
year YM      permno                                                             
2007 2007-01 10025    11903       37172   45.320000      8086.0  3.700557e+05   
             10026    12825       12684   39.689999      7613.0  7.653725e+05   
             10042    12139        4832    0.720000     26008.0  3.598898e+04   
             10078    12136        1719    6.130000  11333293.0  2.390900e+07   
             10104    12142        4413   16.430000   7234361.0  8.892640e+07   
...                     ...         ...         ...         ...           ...   
2021 2021-12 93304   184167       91339   36.750000    183303.0  1.695898e+06   
             93373   184323       74074    3.020000    528712.0  2.065325e+05   
             93374   184899       64442   74.510002    159495.0  8.587073e+06   
             93423    10567       22547   39.490002    254917.0  3.661156e+06   
             93436   184996       24952  936.719971   6361641.0  1.092218e+09   

                       absacc       acc    aeavol       age       agr  ...  \
year YM      permno                                                    ...   
2007 2007-01 10025   0.698728 -0.745547 -0.646819  0.457506 -0.979644  ...   
             10026   0.577608 -0.635623 -0.393384  0.457506  0.118575  ...   
             10042   0.990840 -0.989822 -0.894148  0.457506 -0.989822  ...   
             10078   0.654962 -0.711959 -0.128753  0.905344 -0.147074  ...   
             10104  -0.014758 -0.107379  0.780153  0.905344  0.770992  ...   
...                       ...       ...       ...       ...       ...  ...   
2021 2021-12 93304  -0.855522  0.706269 -0.871045 -0.500896  0.613134  ...   
             93373  -0.213134  0.143881  0.663284 -0.500896 -0.875821  ...   
             93374  -0.514030  0.423284 -0.223881 -0.500896  0.328955  ...   
             93423   0.242985 -0.303881  0.560597  0.143881 -0.584478  ...   
             93436   0.584478 -0.620299 -0.362388 -0.500896  0.856716  ...   

                     sic2_73  sic2_75  sic2_78  sic2_79  sic2_80  sic2_81  \
year YM      permno                                                         
2007 2007-01 10025         0        0        0        0        0        0   
             10026         0        0        0        0        0        0   
             10042         0        0        0        0        0        0   
             10078         0        0        0        0        0        0   
             10104         1        0        0        0        0        0   
...                      ...      ...      ...      ...      ...      ...   
2021 2021-12 93304         0        0        0        0        0        0   
             93373         0        0        0        0        0        0   
             93374         0        0        0        0        0        0   
             93423         0        0        0        1        0        0   
             93436         0        0        0        0        0        0   

                     sic2_82  sic2_83  sic2_87  sic2_99  
year YM      permno                                      
2007 2007-01 10025         0        0        0        0  
             10026         0        0        0        0  
             10042         0        0        0        0  
             10078         0        0        0        0  
             10104         0        0        0        0  
...                      ...      ...      ...      ...  
2021 2021-12 93304         0        0        0        0  
             93373         0        0        0        0  
             93374         0        0        0        0  
             93423         0        0        0        0  
             93436         0        0        0        0  

[338288 rows x 188 columns]

In [4]:
# Select only relevant columns for X and Y (exclude industry & reprisk rating dummy for now)
info_vars = ['YM', 'year', 'permno', 'gvkey', 'reprisk_id', 'prc', 'vol', 'mve_m']
X_vars = ['absacc', 'acc', 'aeavol', 'age', 'agr', 'baspread', 'beta', 'betasq', 'bm', 'bm_ia', 'cash', 'cashdebt',
          'cashpr', 'cfp', 'cfp_ia', 'chatoia', 'chcsho', 'chempia', 'chinv', 'chmom', 'chpmia', 'chtx', 'cinvest',
          'convind', 'currat', 'depr', 'divi', 'divo', 'dolvol', 'dy', 'ear', 'egr', 'ep', 'gma', 'grcapx', 'grltnoa',
          'herf', 'hire', 'idiovol', 'ill', 'indmom', 'invest', 'lev', 'lgr', 'maxret', 'mom12m', 'mom1m', 'mom36m',
          'mom6m', 'ms', 'mve', 'mve_ia', 'nincr', 'operprof', 'orgcap', 'pchcapx_ia', 'pchcurrat', 'pchdepr',
          'pchgm_pchsale', 'pchquick', 'pchsale_pchinvt', 'pchsale_pchrect', 'pchsale_pchxsga', 'pchsaleinv', 'pctacc',
          'pricedelay', 'ps', 'quick', 'rd', 'rd_mve', 'rd_sale', 'realestate', 'retvol', 'roaq', 'roavol', 'roeq',
          'roic', 'rsup', 'salecash', 'saleinv', 'salerec', 'secured', 'securedind', 'sgr', 'sin', 'sp', 'std_dolvol',
          'std_turn', 'stdacc', 'stdcf', 'tang', 'tb', 'turn', 'zerotrade']
sic2_vars = [col for col in df if col.startswith('sic2')]
reprisk_vars = ['country_sector_average', 'country_sector_average_01', 'current_rri', 'current_rri_01',
                'peak_rri', 'peak_rri_01', 'trend_rri', 'trend_rri_01']
reprisk_rating_vars = [col for col in df if col.startswith('reprisk_rating')]
Y_vars = ['ret', 'ret_wins', 'ret_ex']

# X
X = df[X_vars + sic2_vars + reprisk_vars + reprisk_rating_vars]

# Y
Y = df[['ret_ex']]

X.shape, Y.shape

((338288, 180), (338288, 1))

### Hyperparameter Optimization: 4-Fold CV (12y/4 = 3y) and 3y Test

In [5]:
# Training (12y - 80%) and Test set (3y - 20%)
X_trai = X.loc['2007':'2018']
Y_trai = Y.loc['2007':'2018']

X_test = X.loc['2019':'2021']
Y_test = Y.loc['2019':'2021']

In [6]:
# 4-Fold cross validation (9y training and 3y validation)
K_FOLDs = 4
YEARS = list(X_trai.index.unique(level='year').astype('str')) 
TOT = len(YEARS)
TRA = int(TOT* (K_FOLDs-1) / K_FOLDs)
OFF = TOT - TRA

for FOLD in range(K_FOLDs):
    VALI = YEARS[(FOLD*OFF):((FOLD+1)*OFF)]
    TRAI = [x for x in YEARS if x not in VALI]
    print(VALI, TRAI)    

['2007', '2008', '2009'] ['2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018']
['2010', '2011', '2012'] ['2007', '2008', '2009', '2013', '2014', '2015', '2016', '2017', '2018']
['2013', '2014', '2015'] ['2007', '2008', '2009', '2010', '2011', '2012', '2016', '2017', '2018']
['2016', '2017', '2018'] ['2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015']


In [7]:
# Model
model_name = 'NN2_esg'

def create_NN2(l_rate=0.01, l1_pen=0.0001):
    model = Sequential()
    model.add(Dense(32,
                    activation='relu',
                    input_dim=X_trai.shape[1],
                    kernel_regularizer=l1(l1_pen)))
    model.add(Dense(16,
                    activation='relu',
                    kernel_regularizer=l1(l1_pen)))
    model.add(Dense(1))
    model.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate=l_rate))
    return model

In [8]:
# Hyperparameter optimization
Y_val_preds = Y_trai.copy()
Y_test_preds = Y_test.copy()
results = []

earlyStopping  = EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='min')

model_ix = 0
for l1_pen in [0.001, 0.0001, 0.00005, 0.00001]:
        for l_rate in [0.01, 0.005, 0.001]:
            for ensemble in list(range(0,10)):
                print("•", model_ix, ':',  end=' ')
                    
                for FOLD in range(K_FOLDs):
                    VALI = YEARS[(FOLD*OFF):((FOLD+1)*OFF)]
                    TRAI = [x for x in YEARS if x not in VALI]
                        
                    mcp_save = ModelCheckpoint(
                        '%s/%s_%d_%d.h5'%(model_name, model_name, model_ix, FOLD),
                        save_best_only=True, monitor='val_loss', mode='min', save_weights_only=True
                    )

                    reset_seeds(ensemble*10000)
                        
                    NN_model = create_NN2(l_rate, l1_pen)
                        
                    history = NN_model.fit(
                        X_trai.loc[TRAI], Y_trai.loc[TRAI],
                        validation_data=(X_trai.loc[VALI], Y_trai.loc[VALI]),
                        epochs=200, verbose=0, shuffle=True, batch_size=2000,
                        callbacks=[earlyStopping, mcp_save]
                    )
                    tra_loss = np.min(history.history['loss'])
                    val_loss = np.min(history.history['val_loss'])

                    results.append({
                        'model_ix' :model_ix,
                        'l_rate'   :l_rate,
                        'l1_pen'   :l1_pen,
                        'fold'     :FOLD,
                        'tra_loss' :tra_loss,
                        'val_loss' :val_loss
                    })
                    
                    # Calculate validation predictions
                    best_model = create_NN2(l_rate, l1_pen)
                    best_model.load_weights(r'%s/%s_%d_%d.h5'%(model_name, model_name, model_ix, FOLD));
                    Y_val_preds.loc[VALI, '%s_%d'%(model_name, model_ix)] = best_model.predict(X_trai.loc[VALI])
                    
                    # Calculate predictions for test data, if FOLD = 0
                    if FOLD==0:
                        Y_test_preds.loc[['2019','2020', '2021'], '%s_%d'%(model_name, model_ix)] = best_model.predict(X_test)

                model_ix += 1

• 0 : Epoch 8: early stopping
1929/1929 [==============================] - 4s 2ms/step
Epoch 9: early stopping
2193/2193 [==============================] - 5s 2ms/step
Epoch 9: early stopping
2156/2156 [==============================] - 5s 2ms/step
Epoch 7: early stopping
2066/2066 [==============================] - 5s 2ms/step
• 1 : Epoch 8: early stopping
1929/1929 [==============================] - 4s 2ms/step
Epoch 9: early stopping
2193/2193 [==============================] - 5s 2ms/step
Epoch 6: early stopping
2156/2156 [==============================] - 5s 2ms/step
Epoch 9: early stopping
2066/2066 [==============================] - 4s 2ms/step
• 2 : Epoch 8: early stopping
1929/1929 [==============================] - 4s 2ms/step
Epoch 7: early stopping
2193/2193 [==============================] - 5s 2ms/step
Epoch 7: early stopping
2156/2156 [==============================] - 5s 2ms/step
Epoch 7: early stopping
2066/2066 [==============================] - 5s 2ms/step
• 3 : Epoc

1929/1929 [==============================] - 4s 2ms/step
Epoch 13: early stopping
2193/2193 [==============================] - 4s 2ms/step
Epoch 13: early stopping
2156/2156 [==============================] - 4s 2ms/step
Epoch 18: early stopping
2066/2066 [==============================] - 4s 2ms/step
• 22 : Epoch 17: early stopping
1929/1929 [==============================] - 4s 2ms/step
Epoch 16: early stopping
2193/2193 [==============================] - 4s 2ms/step
Epoch 18: early stopping
2156/2156 [==============================] - 4s 2ms/step
Epoch 17: early stopping
2066/2066 [==============================] - 4s 2ms/step
• 23 : Epoch 21: early stopping
1929/1929 [==============================] - 4s 2ms/step
Epoch 21: early stopping
2193/2193 [==============================] - 4s 2ms/step
Epoch 19: early stopping
2156/2156 [==============================] - 5s 2ms/step
Epoch 21: early stopping
2066/2066 [==============================] - 4s 2ms/step
• 24 : Epoch 22: early stop

2066/2066 [==============================] - 5s 2ms/step
• 42 : Epoch 13: early stopping
1929/1929 [==============================] - 4s 2ms/step
Epoch 15: early stopping
2193/2193 [==============================] - 4s 2ms/step
Epoch 18: early stopping
2156/2156 [==============================] - 5s 2ms/step
Epoch 20: early stopping
2066/2066 [==============================] - 4s 2ms/step
• 43 : Epoch 18: early stopping
1929/1929 [==============================] - 4s 2ms/step
Epoch 34: early stopping
2193/2193 [==============================] - 4s 2ms/step
Epoch 18: early stopping
2156/2156 [==============================] - 4s 2ms/step
Epoch 18: early stopping
2066/2066 [==============================] - 4s 2ms/step
• 44 : Epoch 19: early stopping
1929/1929 [==============================] - 4s 2ms/step
Epoch 22: early stopping
2193/2193 [==============================] - 5s 2ms/step
Epoch 19: early stopping
2156/2156 [==============================] - 4s 2ms/step
Epoch 24: early stop

2156/2156 [==============================] - 4s 2ms/step
Epoch 20: early stopping
2066/2066 [==============================] - 5s 2ms/step
• 63 : Epoch 18: early stopping
1929/1929 [==============================] - 4s 2ms/step
Epoch 24: early stopping
2193/2193 [==============================] - 5s 2ms/step
Epoch 21: early stopping
2156/2156 [==============================] - 4s 2ms/step
Epoch 21: early stopping
2066/2066 [==============================] - 4s 2ms/step
• 64 : Epoch 17: early stopping
1929/1929 [==============================] - 4s 2ms/step
Epoch 15: early stopping
2193/2193 [==============================] - 5s 2ms/step
Epoch 18: early stopping
2156/2156 [==============================] - 4s 2ms/step
Epoch 17: early stopping
2066/2066 [==============================] - 5s 2ms/step
• 65 : Epoch 22: early stopping
1929/1929 [==============================] - 4s 2ms/step
Epoch 21: early stopping
2193/2193 [==============================] - 4s 2ms/step
Epoch 21: early stop

2193/2193 [==============================] - 5s 2ms/step
Epoch 52: early stopping
2156/2156 [==============================] - 5s 2ms/step
Epoch 51: early stopping
2066/2066 [==============================] - 5s 2ms/step
• 84 : Epoch 57: early stopping
1929/1929 [==============================] - 4s 2ms/step
Epoch 45: early stopping
2193/2193 [==============================] - 5s 2ms/step
Epoch 57: early stopping
2156/2156 [==============================] - 4s 2ms/step
Epoch 52: early stopping
2066/2066 [==============================] - 4s 2ms/step
• 85 : Epoch 42: early stopping
1929/1929 [==============================] - 4s 2ms/step
Epoch 41: early stopping
2193/2193 [==============================] - 5s 2ms/step
Epoch 58: early stopping
2156/2156 [==============================] - 5s 2ms/step
Epoch 45: early stopping
2066/2066 [==============================] - 5s 2ms/step
• 86 : Epoch 59: early stopping
1929/1929 [==============================] - 4s 2ms/step
Epoch 48: early stop

...

Epoch 40: early stopping
2193/2193 [==============================] - 5s 2ms/step
Epoch 37: early stopping
2156/2156 [==============================] - 5s 2ms/step
Epoch 28: early stopping
2066/2066 [==============================] - 5s 2ms/step
• 119 : Epoch 65: early stopping
   1/1929 [..............................] - ETA: 56s

C:\Users\rafae\AppData\Local\Temp\ipykernel_2760\2779473123.py:48: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  Y_val_preds.loc[VALI, '%s_%d'%(model_name, model_ix)] = best_model.predict(X_trai.loc[VALI])


1929/1929 [==============================] - 5s 2ms/step


C:\Users\rafae\AppData\Local\Temp\ipykernel_2760\2779473123.py:52: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  Y_test_preds.loc[['2019','2020', '2021'], '%s_%d'%(model_name, model_ix)] = best_model.predict(X_test)


Epoch 43: early stopping
2193/2193 [==============================] - 5s 2ms/step
Epoch 48: early stopping
2156/2156 [==============================] - 5s 2ms/step
Epoch 48: early stopping
2066/2066 [==============================] - 5s 2ms/step


In [9]:
# Save Y_val_preds
Y_val_preds.to_csv(r'%s/%s_val_preds.csv'%(model_name, model_name))
Y_val_preds

ret_ex  NN2_esg_0  NN2_esg_1  NN2_esg_2  NN2_esg_3  \
year YM      permno                                                         
2007 2007-01 10025  -0.031894   0.011666   0.006063   0.011684   0.007911   
             10026  -0.042317   0.011666   0.006063   0.011684   0.007911   
             10042  -0.125751   0.011666   0.006066   0.011684   0.007911   
             10078  -0.080607   0.011666   0.006059   0.011684   0.007911   
             10104  -0.046341   0.011666   0.006061   0.011684   0.007911   
...                       ...        ...        ...        ...        ...   
2018 2018-12 93420   0.086508   0.005554   0.004537   0.008348   0.004325   
             93422   0.466817   0.005554   0.004536   0.008348   0.004325   
             93423   0.105036   0.005554   0.004540   0.008348   0.004325   
             93429  -0.048712   0.005554   0.004537   0.008348   0.004325   
             93436  -0.079564   0.005554   0.004537   0.008348   0.004325   

                     NN2_esg_4  NN2_esg_5  NN2_esg_6  NN2_esg_7  NN2_esg_8  \
year YM      permno                                                          
2007 2007-01 10025    0.006436   0.009020   0.008703   0.008940   0.005088   
             10026    0.006436   0.009023   0.008703   0.008937   0.005087   
             10042    0.006436   0.009020   0.008703   0.008940   0.005086   
             10078    0.006436   0.009015   0.008703   0.008938   0.005087   
             10104    0.006436   0.009013   0.008703   0.008940   0.005088   
...                        ...        ...        ...        ...        ...   
2018 2018-12 93420    0.006922   0.006752   0.006231   0.006969   0.010730   
             93422    0.006922   0.006752   0.006231   0.006967   0.010730   
             93423    0.006922   0.006752   0.006232   0.006965   0.010730   
             93429    0.006922   0.006752   0.006236   0.006959   0.010730   
             93436    0.006922   0.006752   0.006235   0.006963   0.010730   

                     ...  NN2_esg_110  NN2_esg_111  NN2_esg_112  NN2_esg_113  \
year YM      permno  ...                                                       
2007 2007-01 10025   ...     0.016098     0.017669     0.009631     0.027925   
             10026   ...     0.008942     0.010883     0.009293     0.013151   
             10042   ...    -0.025225    -0.035010    -0.022063    -0.003215   
             10078   ...     0.010010     0.007480     0.006376     0.013151   
             10104   ...     0.015029     0.013234     0.009847     0.013151   
...                  ...          ...          ...          ...          ...   
2018 2018-12 93420   ...    -0.006488     0.017752    -0.010887     0.014370   
             93422   ...    -0.019117     0.007956     0.016913    -0.012194   
             93423   ...     0.010453    -0.002086     0.002797     0.010569   
             93429   ...     0.010700     0.003399     0.005963     0.009815   
             93436   ...     0.010700    -0.001668     0.005357     0.007096   

                     NN2_esg_114  NN2_esg_115  NN2_esg_116  NN2_esg_117  \
year YM      permno                                                       
2007 2007-01 10025      0.021682     0.022546     0.019406     0.018377   
             10026      0.003620     0.010453     0.004467     0.007610   
             10042     -0.038086    -0.011502    -0.019681    -0.019573   
             10078      0.008577     0.012816     0.009921     0.013220   
             10104      0.016532     0.012422     0.015742     0.010870   
...                          ...          ...          ...          ...   
2018 2018-12 93420     -0.019347     0.012553     0.004488     0.003901   
             93422      0.000309    -0.001040    -0.018191    -0.004372   
             93423     -0.006907     0.002783     0.017746     0.006538   
             93429      0.010585     0.000793     0.011636     0.005975   
             93436     -0.017288     0.004422    -0.013494    -0.

In [10]:
# Save Y_test_preds
Y_test_preds.to_csv(r'%s/%s_test_preds.csv'%(model_name, model_name))
Y_test_preds

ret_ex  NN2_esg_0  NN2_esg_1  NN2_esg_2  NN2_esg_3  \
year YM      permno                                                         
2019 2019-01 10026   0.004225   0.011666   0.006063   0.011684   0.007911   
             10104   0.036026   0.011666   0.006063   0.011684   0.007911   
             10107   0.075381   0.011666   0.006063   0.011684   0.007911   
             10138   0.072777   0.011666   0.006062   0.011684   0.007911   
             10145   0.076596   0.011666   0.006062   0.011684   0.007911   
...                       ...        ...        ...        ...        ...   
2021 2021-12 93304  -0.096386   0.011666   0.006063   0.011684   0.007911   
             93373  -0.019481   0.011666   0.006058   0.011684   0.007911   
             93374  -0.047552   0.011666   0.006062   0.011684   0.007911   
             93423  -0.072569   0.011666   0.006060   0.011684   0.007911   
             93436  -0.113609   0.011666   0.006064   0.011684   0.007911   

                     NN2_esg_4  NN2_esg_5  NN2_esg_6  NN2_esg_7  NN2_esg_8  \
year YM      permno                                                          
2019 2019-01 10026    0.006436   0.009024   0.008703   0.008936   0.005088   
             10104    0.006436   0.009020   0.008703   0.008941   0.005088   
             10107    0.006436   0.009015   0.008703   0.008941   0.005087   
             10138    0.006436   0.009022   0.008703   0.008936   0.005088   
             10145    0.006436   0.009017   0.008703   0.008937   0.005087   
...                        ...        ...        ...        ...        ...   
2021 2021-12 93304    0.006436   0.009023   0.008703   0.008937   0.005089   
             93373    0.006436   0.009024   0.008703   0.008931   0.005087   
             93374    0.006436   0.009020   0.008703   0.008938   0.005087   
             93423    0.006436   0.009027   0.008703   0.008935   0.005087   
             93436    0.006436   0.009011   0.008703   0.008939   0.005087   

                     ...  NN2_esg_110  NN2_esg_111  NN2_esg_112  NN2_esg_113  \
year YM      permno  ...                                                       
2019 2019-01 10026   ...     0.008942     0.010098     0.007950     0.013151   
             10104   ...     0.012335     0.010645     0.000891     0.012755   
             10107   ...     0.010037     0.007897     0.008832     0.013151   
             10138   ...     0.011258     0.009879     0.006535     0.013151   
             10145   ...     0.008942     0.009039     0.006635     0.009726   
...                  ...          ...          ...          ...          ...   
2021 2021-12 93304   ...     0.002992     0.010037     0.007371     0.010392   
             93373   ...    -0.014488    -0.021858    -0.010336    -0.013971   
             93374   ...     0.014683     0.011641     0.007024     0.013151   
             93423   ...     0.013887    -0.000960     0.008000     0.013151   
             93436   ...     0.004580     0.004517     0.010630     0.008646   

                     NN2_esg_114  NN2_esg_115  NN2_esg_116  NN2_esg_117  \
year YM      permno                                                       
2019 2019-01 10026      0.009669     0.010902     0.005462     0.004801   
             10104     -0.001677     0.003820     0.004182    -0.005010   
             10107      0.009956     0.012070     0.006342     0.007986   
             10138      0.011292     0.007917     0.004816     0.006246   
             10145      0.009496     0.009610     0.004838     0.005365   
...                          ...          ...          ...          ...   
2021 2021-12 93304      0.017144     0.004292     0.009497     0.006460   
             93373      0.001835    -0.045553    -0.032068    -0.036441   
             93374      0.009972     0.005831     0.007720     0.006445   
             93423      0.014171     0.013881     0.013902     0.012448   
             93436      0.001656    -0.007373     0.001585    -0.

In [11]:
# Result overview
table = pd.DataFrame(results)
table = table.groupby(['model_ix', 'l_rate', 'l1_pen']).mean().sort_values('val_loss')
table.to_csv(r'%s/%s_results.csv'%(model_name, model_name))

np.sqrt(table[['tra_loss', 'val_loss']].head(20)) * 100

,,,tra_loss,val_loss
model_ix,l_rate,l1_pen,,
54,0.001,0.00010,12.990696,12.953650
51,0.001,0.00010,12.989840,12.954744
81,0.001,0.00005,12.977617,12.955207
59,0.001,0.00010,12.990455,12.956974
89,0.001,0.00005,12.973617,12.958101
55,0.001,0.00010,12.992280,12.960384
85,0.001,0.00005,12.975841,12.961063
91,0.010,0.00001,12.978927,12.961299
83,0.001,0.00005,12.973895,12.965009


## Model Selection: Maximize SR on LS-Portfolio on validation data

In [12]:
# Model
model_name = 'NN2_esg'

# Get predictions
val = pd.read_csv(r'%s/%s_val_preds.csv'%(model_name, model_name), index_col=['year', 'YM', 'permno'])
test = pd.read_csv(r'%s/%s_test_preds.csv'%(model_name, model_name), index_col=['year', 'YM', 'permno'])

# Get overview table
table = pd.read_csv(r'%s/%s_results.csv'%(model_name, model_name)).sort_values(['model_ix'])

# Extract hyperparameters
list_l_rate = list(pd.unique(table['l_rate']))
list_l1_pen = list(pd.unique(table['l1_pen']))

In [13]:
# Loop over hyperparameter combinations to get ensemble predictions for each hyperparameter combination
val_results = []
hyper_comb = 0
for l_rate in list_l_rate:
    for l1_pen in list_l1_pen:
        # Get list of relevant model_ix
        list_model_ix = table[(table['l_rate'] == l_rate) & (table['l1_pen'] == l1_pen)]
        list_model_ix = list(list_model_ix['model_ix'])
        # Get list of column names
        list_col_names = []
        for ele in list_model_ix:
            list_col_names.append('%s_%d'%(model_name, ele))
        # Select relevant returns and average predictions
        val_ret = val[list_col_names]
        val_ret = val_ret.mean(axis=1)
        # Append ensemble return prediction to true returns
        comb = val[['ret_ex']].copy()
        comb['ret_pred'] = val_ret
        comb = comb.reset_index()
        # Sort the data by predicted returns and divide the data into quintiles
        comb['quintile'] = comb.groupby(['YM'])['ret_pred'].transform(lambda x: pd.qcut(x.rank(method='first'), 5, labels=np.arange(1,6)))
        # Calculate the mean return for each YM
        comb_mean = comb.groupby(['YM', 'quintile']).agg(ret_ex = ('ret_ex', 'mean'))
        # Add LS-Strategy
        comb_mean = comb_mean[['ret_ex']].unstack().add_prefix('Q')
        comb_mean.columns = comb_mean.columns.droplevel(0)
        comb_mean.columns.name = None
        comb_mean['LS'] = comb_mean['Q5'] - comb_mean['Q1']
        # Calculate the average return, standard deviation and Sharpe Ratio (annualized) per Quintile
        summary = pd.DataFrame()
        summary['mean'] = comb_mean.mean()
        summary['std'] = comb_mean.std()
        summary['SR'] = summary['mean'] / summary['std'] * np.sqrt(12)
        # Append results
        val_results.append({
            'hyper_comb':hyper_comb,
            'l_rate'    :l_rate,
            'l1_pen'    :l1_pen,
            'SR_Q1'     :summary.loc['Q1','SR'],
            'SR_Q5'     :summary.loc['Q5','SR'],
            'SR_LS'     :summary.loc['LS','SR'],
            'Mean_Q1'   :summary.loc['Q1','mean'],
            'Mean_Q5'   :summary.loc['Q5','mean'],
            'Mean_LS'   :summary.loc['LS','mean'],
        })
        hyper_comb += 1
        
# Save results
val_results = pd.DataFrame(val_results)
val_results.to_csv(r'results/{}_val.csv'.format(model_name))

In [14]:
# Select best hyperparemeters (max. SR_LS)
val_results = val = pd.read_csv(r'results/{}_val.csv'.format(model_name))
val_results = val_results.set_index(['l_rate', 'l1_pen']).sort_values(['SR_LS'], ascending=False)
l_rate_opt, l1_pen_opt = tuple([x for x in val_results.iloc[0].name[0:]])
print('Optimal l_rate:', l_rate_opt)
print('Optimal l1_pen:', l1_pen_opt)
val_results

Optimal l_rate: 0.001
Optimal l1_pen: 1e-05


,,Unnamed: 0,hyper_comb,SR_Q1,SR_Q5,SR_LS,Mean_Q1,Mean_Q5,Mean_LS
l_rate,l1_pen,,,,,,,,
0.001,0.00001,11,11,0.079273,0.418706,0.681920,0.001747,0.007648,0.005902
0.005,0.00001,7,7,0.048775,0.420903,0.616302,0.001112,0.007188,0.006075
0.010,0.00001,3,3,0.031606,0.440019,0.603054,0.000756,0.007318,0.006562
0.001,0.00005,10,10,0.043046,0.538062,0.529550,0.001042,0.007983,0.006942
0.005,0.00005,6,6,0.032779,0.607010,0.491336,0.000808,0.008203,0.007395
0.010,0.00005,2,2,0.076926,0.612532,0.454016,0.001854,0.008230,0.006375
0.001,0.00010,9,9,0.123511,0.595502,0.286055,0.002981,0.007445,0.004463
0.005,0.00010,5,5,0.172966,0.519300,0.198348,0.004120,0.006964,0.002845
0.001,0.00100,8,8,0.304939,0.351587,0.164863,0.005268,0.006696,0.001428


## Out-of-Sample: Get Predictions of model with best hyperparameter combination

In [15]:
# Get list of model_ix's with best hyperparameter combinations and make ensemble predictions
list_model_ix = table[(table['l_rate'] == l_rate_opt) & (table['l1_pen'] == l1_pen_opt)]
list_model_ix = list(list_model_ix['model_ix'])

# Get list of column names
list_col_names = []
for ele in list_model_ix:
    list_col_names.append('%s_%d'%(model_name, ele))
    
# Select relevant returns and average predictions
test_ret = test[list_col_names].copy()
test_ret['y_pred'] = test_ret.mean(axis=1)

# Save predictions
test_ret = test_ret.reset_index()
test_ret = test_ret[['YM', 'permno', 'y_pred']]
test_ret.to_csv(r'results/{}_predictions.csv'.format(model_name), index=False)